<a href="https://colab.research.google.com/github/sanislearning/LangChainProjects/blob/main/SemanticSearchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Goal is to familiarize with LangChain's document loader, embedding and vector store abstractions

In [3]:
import getpass
import os

os.environ["LANGSMITH_TRACING"]='true'
os.environ['LANGSMITH_API_KEY']=getpass.getpass()

··········


In [5]:
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ['GOOGLE_API_KEY']=getpass.getpass("Enter the Gemini API Key")

Enter the Gemini API Key··········


In [6]:
from langchain.chat_models import init_chat_model
llm=init_chat_model("gemini-2.0-flash",model_provider='google_genai')

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings #imports the Embedding model related to the Gemini models
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [10]:
#Creates a vector store in-memory, ie in the form of a python dictionary and uses cosine similarity to match
from langchain_core.vectorstores import InMemoryVectorStore
vector_store=InMemoryVectorStore(embeddings)